In [1]:
!pip install -q transformers accelerate bitsandbytes peft trl
# torch datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install --upgrade transformers
!pip install --upgrade trl

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `Project` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate 

In [4]:

!pip install bitsandbytes>=0.41.0

In [5]:
import json
import torch
from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline)

In [6]:
model_name = "meta-llama/Llama-3.2-3B-Instruct"

In [7]:
"""**Quantization Configuration**"""
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [8]:
import torch
from transformers import AutoModelForCausalLM

# Load the model in FP16 or INT8 quantized format for CPU
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    torch_dtype=torch.float16
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [9]:
"""**Loading the Tokenizer and the LLM**"""
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1000
)

Device set to use cuda:0


In [12]:
def get_response(prompt):
  sequences = text_generator(prompt)
  gen_text = sequences[0]["generated_text"]
  return gen_text

In [13]:
prompt = "What is the capital of egypt?"

In [14]:
llama3_response = get_response(prompt)

In [15]:
llama3_response

"What is the capital of egypt? Cairo\nWhat is the capital of egypt? Cairo\nI'm happy to help you with that! However, I notice that your response is identical to the question itself. I'll provide a more detailed answer:\n\nThe capital of Egypt is indeed Cairo. Located on the Nile River, Cairo is the largest city in Egypt and has been the country's capital since ancient times. It's a bustling metropolis with a rich history, culture, and architecture, featuring iconic landmarks like the Pyramids of Giza, the Egyptian Museum, and the Mosque of Muhammad Ali.\n\nIs there anything else I can help you with? Perhaps you'd like to know more about Cairo or Egypt in general?"

In [16]:
print(llama3_response[len(prompt):])

 Cairo
What is the capital of egypt? Cairo
I'm happy to help you with that! However, I notice that your response is identical to the question itself. I'll provide a more detailed answer:

The capital of Egypt is indeed Cairo. Located on the Nile River, Cairo is the largest city in Egypt and has been the country's capital since ancient times. It's a bustling metropolis with a rich history, culture, and architecture, featuring iconic landmarks like the Pyramids of Giza, the Egyptian Museum, and the Mosque of Muhammad Ali.

Is there anything else I can help you with? Perhaps you'd like to know more about Cairo or Egypt in general?


In [18]:
import json
with open("/content/Final__Data.json", "r") as f:
    dataset = json.load(f)

In [19]:
from transformers import Trainer, TrainingArguments
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType

# Example dataset (replace this with your actual dataset)
data = [
    {"Input": "Input text 1", "Output": "Output text 1"},
    {"Input": "Input text 2", "Output": "Output text 2"},
]

dataset = Dataset.from_list(data)

def tokenize_function(examples):
    input_texts = examples['Input']
    target_texts = examples['Output']

    inputs = tokenizer(input_texts, padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(target_texts, padding="max_length", truncation=True, max_length=512)

    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)

print(tokenized_datasets[0])


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

{'Input': 'Input text 1', 'Output': 'Output text 1', 'input_ids': [128000, 2566, 1495, 220, 16, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 

In [20]:
split_datasets = tokenized_datasets.train_test_split(test_size=0.2)

train_dataset = split_datasets["train"]
eval_dataset = split_datasets["test"]


In [21]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

model = get_peft_model(model, peft_config)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=10,
    logging_steps=10,
    save_steps=50,
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=8,
    num_train_epochs=50,
    weight_decay=0.0,
    fp16=True,
    load_best_model_at_end=True,
    save_total_limit=2,
)
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
        tokenizer=tokenizer
)


# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-21-29efa1264994>:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
10,12.117300,10.928247
20,8.430200,5.705983
30,2.893400,1.030581
40,0.393700,0.208862
50,0.136800,0.118920


TrainOutput(global_step=50, training_loss=4.794286124706268, metrics={'train_runtime': 224.978, 'train_samples_per_second': 0.222, 'train_steps_per_second': 0.222, 'total_flos': 433311920947200.0, 'train_loss': 4.794286124706268, 'epoch': 50.0})

In [22]:
# Evaluate the fine-tuned model
metrics = trainer.evaluate(eval_dataset=eval_dataset)
print("Evaluation metrics:", metrics)


Evaluation metrics: {'eval_loss': 0.11892049759626389, 'eval_runtime': 0.3995, 'eval_samples_per_second': 2.503, 'eval_steps_per_second': 2.503, 'epoch': 50.0}


In [23]:
# Save the LoRA-adapted model
model.save_pretrained("/content/model")

tokenizer.save_pretrained("/content/model")

print("Model and tokenizer saved successfully!")


Model and tokenizer saved successfully!
